# Stock Market Prediction Model of the Magnificient Seven 

#pip install the dependencies from requirements. May take up to 12 minutes

In [7]:
!python3 -m pip install -r requirements.txt


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


### importing our libraries after pipinstall

In [8]:

#python data manipulation
import pandas as pd
import numpy as np

#data visualizaiton tools, EDA
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

#mysql and Cassandra
import mysql.connector                         
from cassandra.cluster import Cluster          

#ARIMA
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm

#LSTM Neural network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import tensorflow as tf

# our finance data
import yfinance as yf



## Extracting the Historical Data.


 ### The historical aspect of the data is starting from January 1st 2020 to yesterday (april 6th), We can adjust this to most recent always. We extracted the 7 companies and start with first 7 features from our project proposal: Date/time, open price, low price, high price, adjusted close price, trading volume

In [9]:
def extract_data(tickers, start_date="2020-01-01", end_date="2025-04-06", interval="1d"):
    # importing the yfinance api data, making sure the autoadjust is off because it overrides the close value if its on
    df = yf.download(tickers, start=start_date, end=end_date, interval=interval, auto_adjust=False)
    # tidying the data to make it better for analysis and transformation later
    df.columns = df.columns.swaplevel(0, 1)
    df = df.sort_index(axis=1, level=0)
    df_flat = df.stack(level=0, future_stack=True).reset_index()  
    df_flat.rename(columns={'level_0': 'Date'}, inplace=True)
    # renaming 'Adj Close' to 'Adj_Close' for consistency
    if 'Adj Close' in df_flat.columns:
        df_flat.rename(columns={'Adj Close': 'Adj_Close'}, inplace=True)
    # adding the previous day's close price per ticker
    df_flat['Previous_Close'] = df_flat.groupby('Ticker')['Close'].shift(1)
    # selecting the columns we expect and need
    expected_cols = ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Adj_Close', 'Previous_Close', 'Volume']
    existing_cols = [col for col in expected_cols if col in df_flat.columns]
    return df_flat[existing_cols]


In [10]:
#giving yfinance a list of companies to return a list of data of. AND displaying the df
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "META", "NVDA", "TSLA"]
df_raw = extract_data(tickers)
display(df_raw)

[*********************100%***********************]  7 of 7 completed


Price,Date,Ticker,Open,High,Low,Close,Adj_Close,Previous_Close,Volume
0,2020-01-02,AAPL,74.059998,75.150002,73.797501,75.087502,72.716080,NaN,135480400
1,2020-01-02,AMZN,93.750000,94.900497,93.207497,94.900497,94.900497,NaN,80580000
2,2020-01-02,GOOGL,67.420502,68.433998,67.324501,68.433998,68.108376,NaN,27278000
3,2020-01-02,META,206.750000,209.789993,206.270004,209.779999,208.795929,NaN,12077100
4,2020-01-02,MSFT,158.779999,160.729996,158.330002,160.619995,153.323257,NaN,22622100
...,...,...,...,...,...,...,...,...,...
9249,2025-04-04,GOOGL,148.009995,151.070007,145.380005,145.600006,145.600006,150.720001,62259500
9250,2025-04-04,META,506.619995,518.000000,494.200012,504.730011,504.730011,531.619995,38589800
9251,2025-04-04,MSFT,364.130005,374.589996,359.480011,359.839996,359.839996,373.109985,49209900
9252,2025-04-04,NVDA,98.910004,100.129997,92.110001,94.309998,94.309998,101.800003,532273800


## Historical Data: Data Cleaning: 
### We will clean the data by taking care of any null values. We can use back and forward filling if there is a value missing from a float/int value. Otherwise we will drop the value if its in the date or Ticker/ or use the average of the past few days for the column. Additionally if there are are any duplicate records for a company and a speciifc date, one should be kept while rest dropped.

In [11]:
nans = df_raw.isna().sum()
print(nans)
def cleaning(df):
    # Step 1: Drop duplicates (keep the first entry for each Ticker-Date pair)
    df = df.drop_duplicates(subset=['Date', 'Ticker'], keep='first')

    # Step 2: Define numeric columns to clean (excluding 'Date' and 'Ticker')
    numeric_cols = ['Open', 'High', 'Low', 'Close', 'Adj_Close', 'Volume', 'Previous_Close']
    
    # Step 3: Fill missing values for numeric columns
    for col in numeric_cols:
        # Forward fill and backward fill first
        df[col] = df.groupby('Ticker')[col].transform(lambda x: x.ffill().bfill())
        
        # Fill remaining NaNs with rolling mean (3-day window)
        df[col] = df.groupby('Ticker')[col].transform(
            lambda x: x.fillna(x.rolling(window=3, min_periods=1).mean())
        )

    # Step 4: Ensure 'Previous_Close' is properly filled (first entry has no previous data)
    df['Previous_Close'] = df.groupby('Ticker')['Previous_Close'].ffill()

    # Step 5: Drop rows with nulls in 'Date' or 'Ticker' (shouldn't exist, but safety check)
    df = df.dropna(subset=['Date', 'Ticker'])

    return df
# Apply cleaning
df_cleaned = cleaning(df_raw)

# Verify results
print("Null values after cleaning:")
print(df_cleaned.isna().sum())
display(df_cleaned)

Price
Date              0
Ticker            0
Open              0
High              0
Low               0
Close             0
Adj_Close         0
Previous_Close    7
Volume            0
dtype: int64
Null values after cleaning:
Price
Date              0
Ticker            0
Open              0
High              0
Low               0
Close             0
Adj_Close         0
Previous_Close    0
Volume            0
dtype: int64


Price,Date,Ticker,Open,High,Low,Close,Adj_Close,Previous_Close,Volume
0,2020-01-02,AAPL,74.059998,75.150002,73.797501,75.087502,72.716080,75.087502,135480400
1,2020-01-02,AMZN,93.750000,94.900497,93.207497,94.900497,94.900497,94.900497,80580000
2,2020-01-02,GOOGL,67.420502,68.433998,67.324501,68.433998,68.108376,68.433998,27278000
3,2020-01-02,META,206.750000,209.789993,206.270004,209.779999,208.795929,209.779999,12077100
4,2020-01-02,MSFT,158.779999,160.729996,158.330002,160.619995,153.323257,160.619995,22622100
...,...,...,...,...,...,...,...,...,...
9249,2025-04-04,GOOGL,148.009995,151.070007,145.380005,145.600006,145.600006,150.720001,62259500
9250,2025-04-04,META,506.619995,518.000000,494.200012,504.730011,504.730011,531.619995,38589800
9251,2025-04-04,MSFT,364.130005,374.589996,359.480011,359.839996,359.839996,373.109985,49209900
9252,2025-04-04,NVDA,98.910004,100.129997,92.110001,94.309998,94.309998,101.800003,532273800


## Historical Data, Feature Engineering:

### We are going to be transforming the tidy dataframe by doing some feature engineering. We will create the following fields into our table by manipulating the prexisting data from df_cleaned: 
1. Simple Moving Average (SMA)
2. Exponential Moving Average (EMA)
3. Relative Strength Index (RSI)
4. Bollinger Bands
5. MACD (Moving Average Convergence Divergence)
6. On-Balance Volume (OBV)
7. Volatility (ATR - Average True Range)


In [12]:
def transform(df):
    # Ensure data is sorted by Ticker and Date
    df = df.sort_values(['Ticker', 'Date']).reset_index(drop=True)
    
    # 1. Simple Moving Average (SMA)
    #uses 20 periods of data (NaN till 19 row)
    df['SMA_20'] = df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(window=20).mean())
    
    # 2. Exponential Moving Average (EMA)
    df['EMA_20'] = df.groupby('Ticker')['Close'].transform(lambda x: x.ewm(span=20, adjust=False).mean())
    
    # 3. Relative Strength Index (RSI)
    #uses 14 periods of data (NaN till 13 row)
    def compute_rsi(data, period=14):
        delta = data.diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(window=period).mean()
        avg_loss = loss.rolling(window=period).mean()
        rs = avg_gain / avg_loss
        return 100 - (100 / (1 + rs))
    df['RSI_14'] = df.groupby('Ticker')['Close'].transform(compute_rsi)
    
    # 4. Bollinger Bands
    #NaN till we get SMA values (19th row)
    df['BB_Middle'] = df['SMA_20']
    df['BB_Std'] = df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(window=20).std())
    df['BB_Upper'] = df['BB_Middle'] + 2 * df['BB_Std']
    df['BB_Lower'] = df['BB_Middle'] - 2 * df['BB_Std']
    
    # 5. MACD
    def compute_macd(data):
        ema12 = data.ewm(span=12, adjust=False).mean()
        ema26 = data.ewm(span=26, adjust=False).mean()
        return ema12 - ema26
    df['MACD'] = df.groupby('Ticker')['Close'].transform(compute_macd)
    df['MACD_Signal'] = df.groupby('Ticker')['MACD'].transform(lambda x: x.ewm(span=9, adjust=False).mean())
    
    # # 6. On-Balance Volume (OBV)
    # def compute_obv(group):
    #     obv = (np.where(group['Close'].diff() > 0, group['Volume'],
    #             np.where(group['Close'].diff() < 0, -group['Volume'], 0))).cumsum()
    #     return obv
    
    # df['OBV'] = df.groupby('Ticker', group_keys=False).apply(compute_obv).reset_index(drop=True)
    
    # # 7. Average True Range (ATR)
    # def compute_atr(group):
    #     high_low = group['High'] - group['Low']
    #     high_close = (group['High'] - group['Close'].shift()).abs()
    #     low_close = (group['Low'] - group['Close'].shift()).abs()
    #     tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    #     return tr.rolling(window=14).mean()
    
    # df['ATR_14'] = df.groupby('Ticker', group_keys=False).apply(compute_atr).reset_index(drop=True)
    
    # # Forward fill missing values created by rolling windows
    # df = df.groupby('Ticker').ffill().reset_index(drop=True)
    
    return df

# Apply transformation to cleaned data
df_transformed = transform(df_cleaned)
display(df_transformed[['Date', 'Ticker', 'SMA_20', 'EMA_20', 'RSI_14', 'BB_Upper', 'BB_Lower', 'MACD']].head(n=20)) #only shows APPL b/c all other tickers appear later. 
#add in 'OBV' and 'ATR_14' when fixed 

#to filter a specific ticker: 
display(df_transformed[df_transformed['Ticker'] == 'MSFT'].iloc[20:25])

#to shuffle/show random rows: 
display(df_transformed.sample(10))  # Show random rows

# count = len(df_transformed)
# print(count)


Price,Date,Ticker,SMA_20,EMA_20,RSI_14,BB_Upper,BB_Lower,MACD
0,2020-01-02,AAPL,NaN,75.087502,NaN,NaN,NaN,0.000000
1,2020-01-03,AAPL,NaN,75.017977,NaN,NaN,NaN,-0.058234
2,2020-01-06,AAPL,NaN,75.011503,NaN,NaN,NaN,-0.055930
3,2020-01-07,AAPL,NaN,74.972075,NaN,NaN,NaN,-0.081607
4,2020-01-08,AAPL,NaN,75.050687,NaN,NaN,NaN,-0.005068
5,2020-01-09,AAPL,NaN,75.275145,NaN,NaN,NaN,0.183389
6,2020-01-10,AAPL,NaN,75.494893,NaN,NaN,NaN,0.342911
7,2020-01-13,AAPL,NaN,75.851570,NaN,NaN,NaN,0.596207
8,2020-01-14,AAPL,NaN,76.072372,NaN,NaN,NaN,0.702507
9,2020-01-15,AAPL,NaN,76.240241,NaN,NaN,NaN,0.751062


Price,Date,Ticker,Open,High,Low,Close,Adj_Close,Previous_Close,Volume,SMA_20,EMA_20,RSI_14,BB_Middle,BB_Std,BB_Upper,BB_Lower,MACD,MACD_Signal
5308,2020-01-31,MSFT,172.210007,172.399994,169.580002,170.229996,162.496704,172.779999,36142700,164.167999,165.244291,65.892017,164.167999,3.935323,172.038644,156.297353,2.175122,1.508417
5309,2020-02-03,MSFT,170.429993,174.500000,170.399994,174.380005,166.458160,170.229996,30107000,164.955999,166.114359,68.389658,164.955999,4.324559,173.605117,156.306881,2.627071,1.732148
5310,2020-02-04,MSFT,177.139999,180.639999,176.309998,180.119995,171.937393,174.380005,36433300,166.010499,167.448229,75.869974,166.010499,5.271194,176.552887,155.468110,3.409114,2.067541
5311,2020-02-05,MSFT,184.029999,184.199997,178.410004,179.899994,171.727417,180.119995,39186300,167.126498,168.634112,74.631681,167.126498,5.734751,178.596000,155.656997,3.965426,2.447118
5312,2020-02-06,MSFT,180.970001,183.820007,180.059998,183.630005,175.287979,179.899994,27751400,168.303499,170.062292,75.172992,168.303499,6.569494,181.442486,155.164511,4.653644,2.888423


Price,Date,Ticker,Open,High,Low,Close,Adj_Close,Previous_Close,Volume,SMA_20,EMA_20,RSI_14,BB_Middle,BB_Std,BB_Upper,BB_Lower,MACD,MACD_Signal
5767,2021-11-24,MSFT,336.279999,338.160004,333.910004,337.910004,328.708740,337.679993,21661300,335.617998,332.817914,52.667877,335.617998,4.419983,344.457964,326.778031,7.429519,8.436332
5026,2024-03-20,META,499.500000,508.200012,495.170013,505.519989,503.682495,496.239990,11711100,493.580998,488.611682,55.940703,493.580998,8.714835,511.010667,476.151329,12.766924,15.730721
3251,2022-05-31,GOOGL,112.746498,115.735001,112.078499,113.762001,113.220688,112.316498,50012000,113.024675,113.402923,49.016354,113.024675,4.018621,121.061917,104.987433,-3.589470,-4.199021
9060,2024-06-27,TSLA,195.169998,198.720001,194.050003,197.419998,197.419998,196.369995,72746500,181.058499,183.157062,67.530599,181.058499,6.978419,195.015337,167.101661,4.276338,2.536693
2052,2022-11-23,AMZN,93.239998,94.580002,92.830002,94.129997,94.129997,93.199997,59414700,95.670000,97.903480,57.530390,95.670000,5.782209,107.234418,84.105583,-4.483151,-5.113075
1949,2022-06-29,AMZN,107.379997,110.989998,106.910004,108.919998,108.919998,107.400002,66375300,113.195750,111.795999,38.124753,113.195750,7.622672,128.441094,97.950406,-2.545193,-3.117335
7150,2022-02-23,NVDA,23.802000,24.155001,22.301001,22.386999,22.347773,23.389999,566511000,24.385100,24.672025,39.420436,24.385100,1.352770,27.090639,21.679561,-0.630489,-0.551063
2664,2020-01-31,GOOGL,73.392998,73.483498,71.352997,71.639000,71.298134,72.712502,43822000,71.976675,71.890864,51.082410,71.976675,1.644811,75.266298,68.687053,0.876712,1.057401
4557,2022-05-06,META,207.339996,209.380005,201.020004,203.770004,202.814148,208.279999,34747200,204.324501,206.141897,47.373950,204.324501,14.191445,232.707392,175.941610,-2.035626,-4.059851
7129,2022-01-24,NVDA,22.330000,23.379999,20.888000,23.372000,23.331051,23.374001,913982000,27.565200,26.795246,16.704481,27.565200,2.292969,32.151137,22.979263,-1.430673,-0.929374
